# Functions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib.ticker import MaxNLocator
import math
from scipy import signal
from scipy.fftpack import fft
from scipy import ndimage
from scipy.io import loadmat
import os
%matplotlib inline

In [ ]:
import import_ipynb
os.chdir('C:\\Users\\hydro_leo\\Documents\\GitHub\\python-code\\Modules')
from time_shifted_PID import Histo3D2Dict
from time_shifted_PID import PIfunc

# Main

In [5]:
figmap = [[np.nan, 7,15,23,31,39,47,np.nan],
          [     1, 8,16,24,32,40,48,    55],
          [     2, 9,17,25,33,41,49,    56],
          [     3,10,18,26,34,42,50,    57],
          [     4,11,19,27,35,43,51,    58],
          [     5,12,20,28,36,44,52,    59],
          [     6,13,21,29,37,45,53,    60],
          [np.nan,14,22,30,38,46,54,np.nan]]
figmap = np.array(figmap)

In [6]:
import warnings
warnings.simplefilter('ignore')

for Fc in [4,7,10]:
    exp_folder = 'D:\\GoogleDrive\\retina\\Chou\'s data\\20200408\\'
    filename = '20200408_OU_cutoff='+str(Fc)+'_sort_unit2'
    annots = loadmat(exp_folder+filename)
    dt = 0.01
    sampling_rate = 20000
    TimeStamps =np.round(np.squeeze(annots['TimeStamps']))
    TimeStamps = TimeStamps.astype(int)
    x = annots['a_data'][0, TimeStamps[0]*sampling_rate:TimeStamps[1]*sampling_rate+1]
    x = x[::int(sampling_rate*dt)]
    x = x.astype(float)
    v = ndimage.gaussian_filter1d(x.copy(), sigma=5, order=1, mode='reflect')/dt
    T=np.arange(len(x))*dt
    u1annots = loadmat(exp_folder+filename[:-1]+'1')

    xstate, _= EqualState(x, 6)
    vstate, _= EqualState(v, 6)
    PIfig, PIax = plt.subplots(nrows=8, ncols=8)
    MIfig, MIax = plt.subplots(nrows=8, ncols=8)
    for channelnumber in np.arange(1,61):
        u2state, _ = np.histogram(annots['Spikes'][0][channelnumber-1]-TimeStamps[0], np.append(0,T))
        u1state, _ = np.histogram(u1annots['Spikes'][0][channelnumber-1]-TimeStamps[0], np.append(0,T))
        if sum(u2state)/len(u2state)/dt <= 0.3 : continue
        window = [-1,1] # second
        timeshift, Information = PIfunc(xstate, u1state, u2state, dt, window)
        timeshift = np.flip(timeshift)
        if max(Information[('Beer','UIv')]) < 0.05: continue
        PIax[np.where(figmap == channelnumber)][0].plot(timeshift,Information[('Beer','UIx')],'r-')
        PIax[np.where(figmap == channelnumber)][0].plot(timeshift,Information[('Beer','UIv')],'b-')
        PIax[np.where(figmap == channelnumber)][0].plot(timeshift,Information[('Beer','Syn')],'k-')
        PIax[np.where(figmap == channelnumber)][0].plot(timeshift,Information[('Beer','Red')],'g-')
        PIax[np.where(figmap == channelnumber)][0].set_title(str(channelnumber))
        PIax[np.where(figmap == channelnumber)][0].grid()

        MIax[np.where(figmap == channelnumber)][0].plot(timeshift,Information[('Beer','UIx')]+Information[('Beer','Red')],'r-')
        MIax[np.where(figmap == channelnumber)][0].plot(timeshift,Information[('Beer','UIv')]+Information[('Beer','Red')],'b-')
        MIax[np.where(figmap == channelnumber)][0].plot(timeshift,Information[('Beer','UIv')]+Information[('Beer','UIx')]+2*Information[('Beer','Red')],'m-')
        MIax[np.where(figmap == channelnumber)][0].plot(timeshift,Information[('Beer','UIv')]+Information[('Beer','UIx')]+Information[('Beer','Red')]+Information[('Beer','Syn')],'k-')
        MIax[np.where(figmap == channelnumber)][0].set_title(str(channelnumber))
        MIax[np.where(figmap == channelnumber)][0].grid()
    PIfig.set_size_inches(60, 30)
    PIfig.savefig(exp_folder+'PID-fig//PI-'+filename[:-5]+'u1vu2.tif', bbox_inches = 'tight',pad_inches = 0.3)
    plt.close(PIfig)
    MIfig.set_size_inches(60, 30)
    MIfig.savefig(exp_folder+'PID-fig//MI-'+filename[:-5]+'u1vu2.tif', bbox_inches = 'tight',pad_inches = 0.3)
    plt.close(MIfig)